#<b>What is Conditional Random Field?</b>

“An undirected graphical model to find pattern recognition in structured prediction ”



1.   CRF is an undirected discriminative probabilistic graphical model
2.   No natural condition between the nodes
3.   Conditional probabilities are replaced by affinity or factor (when the neighbor node is known)
4.  It is cyclic in nature
Always depends on the  immediate neighbors


image.png







#<b> NAME ENTITY RECOGNITION USING CONDITIONAL RANDOM FIELD
Name Entity extraction, also known as entity name extraction or named entity recognition, is an information extraction technique that refers to the process of identifying and classifying key elements from text into pre-defined categories


image.png

#<b> Importing Dataset

In [ ]:
!wget https://www.dropbox.com/s/qtbih1i6x2hrejr/entity-annotated-corpus.zip

--2021-08-22 15:55:49--  https://www.dropbox.com/s/qtbih1i6x2hrejr/entity-annotated-corpus.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/qtbih1i6x2hrejr/entity-annotated-corpus.zip [following]
--2021-08-22 15:55:49--  https://www.dropbox.com/s/raw/qtbih1i6x2hrejr/entity-annotated-corpus.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd9fded271f2d2a320ffebe689b.dl.dropboxusercontent.com/cd/0/inline/BUsXwjNut0YLsVZcfu7E85U9ZTYa8fB9mEWVzu_AHUkVnt49P62hdcp4i7lgG6LORklcwMs7OUFhc7DDOOxRILR_vb-3GykNR-xFSC0xbzDSPI1VNysa5AV1D1Zy1ZtmJ0Cp93ZZvpWslln_zju65mDX/file# [following]
--2021-08-22 15:55:49--  https://ucd9fded271f2d2a320ffebe689b.dl.dropboxusercontent.com/cd/0/inline/BUsXwjNut0YLsVZcfu7E85U9ZTYa8fB9mEWVzu_AHUkVn

#<b> Installing and Importing Libraries

In [ ]:
! pip install eli5

     |████████████████████████████████| 106 kB 5.2 MB/s 


In [ ]:
! pip install sklearn_crfsuite

     |████████████████████████████████| 743 kB 5.1 MB/s 


#<b> Unzip the dataset

image.png

In [ ]:
!unzip entity-annotated-corpus.zip

Archive:  entity-annotated-corpus.zip
  inflating: ner.csv                 
  inflating: ner_dataset.csv         


In [ ]:
import pandas as pds
import eli5 as el
from sklearn_crfsuite import CRF as crf
from sklearn.model_selection import cross_val_predict as cvp
from sklearn_crfsuite.metrics import flat_classification_report as fcr

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


#Load the dataset. 
<b>Origin of the dataset: </b>The dataset has been downloaded from Kaggle <br> https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus
<br>
<b> Dataset descriptions:</b>
This dataset has text with tags that represent grography, persons, organisations etc. 

Let's consider only 10000 rows from this dataset as this is a huge dataset and for computational considerations

### Essential info about entities:
*   geo = Geographical Entity
*   org = Organization
*   gpe = Geopolitical Entity
*   per = Person
*   gpe = Geopolitical Entity
*   tim = Time indicator
*   art = Artifact
*   eve = Event
*   nat = Natural Phenomenon

#### **Inside–outside–beginning (tagging)**
*    The IOB (short for inside, outside, beginning) is a common tagging format for tagging tokens.
*   I- prefix before a tag indicates that the tag is inside a chunk.
*   B- prefix before a tag indicates that the tag is the beginning of a chunk.
*   An O tag indicates that a token belongs to no chunk (outside).

In [ ]:
text_df = pds.read_csv("ner_dataset.csv", encoding="latin1")
text_df=text_df.head(10000)

In [ ]:
text_df.head()

Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O

#<b> Data pre-processing step
Fill the data with NA wherever applicable 


In [ ]:
text_df = text_df.fillna(method="ffill")

In [ ]:
text_df.head()

Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1  Sentence: 1             of   IN   O
2  Sentence: 1  demonstrators  NNS   O
3  Sentence: 1           have  VBP   O
4  Sentence: 1        marched  VBN   O

# <b>Create the list of words with the column 'Word' from the dataset


In [ ]:
text_df["Word"].values

array(['Thousands', 'of', 'demonstrators', ..., 'tribunal', 'in', 'The'],
      dtype=object)

<b> Using set function making distinct word list

In [ ]:
set(text_df["Word"].values)

{'Business',
 'Major',
 'deny',
 'larger',
 'remarks',
 'Union',
 'threatened',
 'downplaying',
 'practitioners',
 'murder',
 'Sutham',
 'additional',
 'rose',
 'guerrillas',
 'Earlier',
 'today',
 'Catholic',
 'option',
 'organizers',
 'dust',
 'return',
 'rocks',
 'vital',
 'interviewer',
 'weekly',
 'stress',
 'may',
 'agreed',
 'During',
 'Muslims',
 'reports',
 'suspended',
 'range',
 'pardoning',
 'shortage',
 'he',
 'orbits',
 'coast',
 'eve',
 'killed',
 'meeting',
 'searchers',
 'Iranian',
 'dollar',
 'If',
 'emperor',
 '16-year-old',
 'violent',
 'human',
 'Labor',
 'ranked',
 'precursor',
 'communication',
 'regret',
 'remains',
 'communities',
 'Obama',
 'waiting',
 'change',
 'carried',
 'host',
 'TamilNet',
 'released',
 'developing',
 'reached',
 'agencies',
 'contact',
 'blew',
 'cleric',
 'unclear',
 'counterpart',
 'previous',
 'indication',
 'Ocean',
 'criticized',
 'operated',
 'television',
 'extending',
 'Dianne',
 'fighting',
 'six-year',
 'Dragan',
 'II',
 'requ

In [ ]:
list_of_words = list(set(text_df["Word"].values))

list_of_words

['Business',
 'Major',
 'deny',
 'larger',
 'remarks',
 'Union',
 'threatened',
 'downplaying',
 'practitioners',
 'murder',
 'Sutham',
 'additional',
 'rose',
 'guerrillas',
 'Earlier',
 'today',
 'Catholic',
 'option',
 'organizers',
 'dust',
 'return',
 'rocks',
 'vital',
 'interviewer',
 'weekly',
 'stress',
 'may',
 'agreed',
 'During',
 'Muslims',
 'reports',
 'suspended',
 'range',
 'pardoning',
 'shortage',
 'he',
 'orbits',
 'coast',
 'eve',
 'killed',
 'meeting',
 'searchers',
 'Iranian',
 'dollar',
 'If',
 'emperor',
 '16-year-old',
 'violent',
 'human',
 'Labor',
 'ranked',
 'precursor',
 'communication',
 'regret',
 'remains',
 'communities',
 'Obama',
 'waiting',
 'change',
 'carried',
 'host',
 'TamilNet',
 'released',
 'developing',
 'reached',
 'agencies',
 'contact',
 'blew',
 'cleric',
 'unclear',
 'counterpart',
 'previous',
 'indication',
 'Ocean',
 'criticized',
 'operated',
 'television',
 'extending',
 'Dianne',
 'fighting',
 'six-year',
 'Dragan',
 'II',
 'requ

#<b> Count the number of distinct words

In [ ]:
num_words = len(list_of_words)
print(num_words)

2746


In [ ]:
text_df.head()

Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1  Sentence: 1             of   IN   O
2  Sentence: 1  demonstrators  NNS   O
3  Sentence: 1           have  VBP   O
4  Sentence: 1        marched  VBN   O

In [ ]:
text_df.groupby("Tag").size().reset_index(name="counts")

Tag  counts
0   B-art      28
1   B-eve      10
2   B-geo     244
3   B-gpe     303
4   B-nat       5
5   B-org     176
6   B-per     160
7   B-tim     149
8   I-art      20
9   I-eve      10
10  I-geo      31
11  I-gpe      20
12  I-nat       2
13  I-org     140
14  I-per     206
15  I-tim      13
16      O    8483

In [ ]:
text_df.shape

(10000, 4)

#**This class is defined to get sentences with POS and tags**

In [ ]:
class get_tokenised_text(object):
    
    def __init__(self, df):
        self.index = 1
        self.df = df
        self.nulls = False
        # Below function is used to concatenate or aggregare word, pos, tag in same tuple. For eg.: ('pope','NN','O')
        concat_function = lambda text: [(word, pos, tag) for word, pos, tag in zip(text["Word"].values.tolist(),
                                                           text["POS"].values.tolist(),
                                                           text["Tag"].values.tolist())]
        # Then group the sentence with the concat function written above just to separate sentences
        self.grouped_sentence = self.df.groupby("Sentence #").apply(concat_function)
        self.full_text = [text for text in self.grouped_sentence]  # get only text as list inside a list
    
    def get_text(self):
        try:
            text = self.df[self.df["Sentence #"] == "Sentence: {}".format(self.index)]
            self.index += 1
            return text["Word"].values.tolist(), text["POS"].values.tolist(), text["Tag"].values.tolist()    
        except:
            self.nulls = True
            return None, None, None
    

#<b>The below function returns the whole text with POS and tags in a list format


In [ ]:
text_df.head()

Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1  Sentence: 1             of   IN   O
2  Sentence: 1  demonstrators  NNS   O
3  Sentence: 1           have  VBP   O
4  Sentence: 1        marched  VBN   O

In [ ]:
# Below function is used to concatenate or aggregare word, pos, tag in same tuple. For eg.: ('pope','NN','O')
concat_function = lambda text: [(word, pos, tag) for word, pos, tag in zip(text_df["Word"].values.tolist(),
                                                    text_df["POS"].values.tolist(),
                                                    text_df["Tag"].values.tolist())]
# Then group the sentence with the concat function written above just to separate sentences
grouped_sentence = text_df.groupby("Sentence #").apply(concat_function)
full_text = [text for text in grouped_sentence]  # get only text as list inside a list

In [ ]:
grouped_sentence # pandas series

Sentence #
Sentence: 1      [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Sentence: 10     [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Sentence: 100    [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Sentence: 101    [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Sentence: 102    [(Thousands, NNS, O), (of, IN, O), (demonstrat...
                                       ...                        
Sentence: 95     [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Sentence: 96     [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Sentence: 97     [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Sentence: 98     [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Sentence: 99     [(Thousands, NNS, O), (of, IN, O), (demonstrat...
Length: 457, dtype: object

In [ ]:
full_text # python list

In [ ]:
text_df_interim = get_tokenised_text(text_df)
print(text_df_interim)

#<b>Converted Text to tokenized text

In [ ]:
tokenised_text = text_df_interim.full_text
print(tokenised_text)

[[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')], [('Iranian', 'JJ', 'B-gpe'), ('officials', 'NNS', 'O'), ('say', 'VBP', 'O'), ('they', 'PRP', 'O'), ('expect', 'VBP', 'O'), ('to', 'TO', 'O'), ('get', 'VB', 'O'), ('access', 'NN', 'O'), ('to', 'TO', 'O'), ('sealed', 'JJ', 'O'), ('sensitive', 'JJ', 'O'), ('parts', 'NNS', 'O'), ('of', 'IN', 'O'), ('the', 'DT', 'O'), ('plant', 'NN', 'O'), ('Wednesday', 'NNP', 'B-tim'), (',', ',', 'O'), ('after', 'IN', 'O'), ('an', 'DT', 'O'), ('IAEA', 'NNP', 'B-org'

In [ ]:
text_df_interim.get_text()

(['Families',
  'of',
  'soldiers',
  'killed',
  'in',
  'the',
  'conflict',
  'joined',
  'the',
  'protesters',
  'who',
  'carried',
  'banners',
  'with',
  'such',
  'slogans',
  'as',
  '"',
  'Bush',
  'Number',
  'One',
  'Terrorist',
  '"',
  'and',
  '"',
  'Stop',
  'the',
  'Bombings',
  '.',
  '"'],
 ['NNS',
  'IN',
  'NNS',
  'VBN',
  'IN',
  'DT',
  'NN',
  'VBD',
  'DT',
  'NNS',
  'WP',
  'VBD',
  'NNS',
  'IN',
  'JJ',
  'NNS',
  'IN',
  '``',
  'NNP',
  'NN',
  'CD',
  'NN',
  '``',
  'CC',
  '``',
  'VB',
  'DT',
  'NNS',
  '.',
  '``'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-per',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'])

#<b> Function to convert text into features. </b>
<br>
Features like , , or 

1.   word parts
2.   Whether it is title
3.   Digit
4.   Upper case or a lower case 

We convert them into sklearn-crfsuite format so that it becomes easy for the package to read the data 
 <br>

<b>Each sentence is converted to a list of dicts

In [ ]:
tokenised_text[0][0]

('Thousands', 'NNS', 'O')

In [ ]:
w= "Thousands"
w[-3:]

'nds'

In [ ]:
w[-2:]

'ds'

In [ ]:
tokenised_text[0][0][1]

'NNS'

In [ ]:
def text2features(text, index):
    word = text[index][0]
    pos_tag = text[index][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(), # starts with Upper case
        'word.isdigit()': word.isdigit(),
        'pos_tag': pos_tag,
        'pos_tag[:2]': pos_tag[:2],
    }
    if index > 0:
        word_next = text[index-1][0]
        pos_tag_next = text[index-1][1]
        features.update({
            '-1:word.lower()': word_next.lower(),
            '-1:word.istitle()': word_next.istitle(),
            '-1:word.isupper()': word_next.isupper(),
            '-1:pos_tag': pos_tag_next,
            '-1:pos_tag[:2]': pos_tag_next[:2],
        })
    else:
        features['BOS'] = True

    if index < len(text)-1:
        word_next = text[index+1][0]
        pos_tag_next = text[index+1][1]
        features.update({
            '+1:word.lower()': word_next.lower(),
            '+1:word.istitle()': word_next.istitle(),
            '+1:word.isupper()': word_next.isupper(),
            '+1:pos_tag': pos_tag_next,
            '+1:pos_tag[:2]': pos_tag_next[:2],
        })
    else:
        features['EOS'] = True

    return features

In [ ]:
tokenised_text # 1st list --> 1st sentece --> (list of lists) - tuple (Word,POS,TAG)

[[('Thousands', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('demonstrators', 'NNS', 'O'),
  ('have', 'VBP', 'O'),
  ('marched', 'VBN', 'O'),
  ('through', 'IN', 'O'),
  ('London', 'NNP', 'B-geo'),
  ('to', 'TO', 'O'),
  ('protest', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('war', 'NN', 'O'),
  ('in', 'IN', 'O'),
  ('Iraq', 'NNP', 'B-geo'),
  ('and', 'CC', 'O'),
  ('demand', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('withdrawal', 'NN', 'O'),
  ('of', 'IN', 'O'),
  ('British', 'JJ', 'B-gpe'),
  ('troops', 'NNS', 'O'),
  ('from', 'IN', 'O'),
  ('that', 'DT', 'O'),
  ('country', 'NN', 'O'),
  ('.', '.', 'O')],
 [('Iranian', 'JJ', 'B-gpe'),
  ('officials', 'NNS', 'O'),
  ('say', 'VBP', 'O'),
  ('they', 'PRP', 'O'),
  ('expect', 'VBP', 'O'),
  ('to', 'TO', 'O'),
  ('get', 'VB', 'O'),
  ('access', 'NN', 'O'),
  ('to', 'TO', 'O'),
  ('sealed', 'JJ', 'O'),
  ('sensitive', 'JJ', 'O'),
  ('parts', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('the', 'DT', 'O'),
  ('plant', 'NN', 'O'),
  ('Wednesday', 'NNP', 'B-tim'),
  ('

In [ ]:
def text_to_features_all(text):
    return [text2features(text, index) for index in range(len(text))]

In [ ]:
def text_to_labels(text):
    return [label for token, postag, label in text]

#<b>

#<b>1.   X will contain all the features data like POS, upper or lower or digit etc. 
#<b>2.  Y is the output variable i.e. the entity




In [ ]:
type(tokenised_text)

list

In [ ]:
tokenised_text[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [ ]:
X = [text_to_features_all(text) for text in tokenised_text]
X

In [ ]:
X[0]

In [ ]:
y = [text_to_labels(text) for text in tokenised_text]
y

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-org',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'I-geo',
  'O'],
 ['O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'B-org', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O'],
 ['O',
  'B-tim',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [ ]:
X[0][0]

{'+1:pos_tag': 'IN',
 '+1:pos_tag[:2]': 'IN',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'of',
 'BOS': True,
 'bias': 1.0,
 'pos_tag': 'NNS',
 'pos_tag[:2]': 'NN',
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'thousands',
 'word[-2:]': 'ds',
 'word[-3:]': 'nds'}

#<b> Building CRF model

In [ ]:
# Syntax to use the CRF model
crf_model = crf(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)
crf_model.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

#<b>Prediction

In [ ]:
# Predict the model and check for accuracy and other metrics
prediction = cvp(estimator=crf_model, X=X, y=y, cv=5)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# <b>Print the output and check for the metrics. How the classification has been done using CRF


In [ ]:
classification_analysis = fcr(y_pred=prediction, y_true=y)
print(classification_analysis)

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        28
       B-eve       0.00      0.00      0.00        10
       B-geo       0.70      0.76      0.73       244
       B-gpe       0.72      0.76      0.74       303
       B-nat       1.00      0.20      0.33         5
       B-org       0.66      0.63      0.64       176
       B-per       0.78      0.82      0.80       160
       B-tim       0.98      0.90      0.94       149
       I-art       0.00      0.00      0.00        20
       I-eve       0.00      0.00      0.00        10
       I-geo       0.73      0.35      0.48        31
       I-gpe       0.73      0.40      0.52        20
       I-nat       0.00      0.00      0.00         2
       I-org       0.66      0.74      0.69       140
       I-per       0.84      0.96      0.90       206
       I-tim       0.90      0.69      0.78        13
           O       0.99      1.00      1.00      8483

    accuracy              

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#<b>Let's check what are the most frequently occured transitions and rarely occured transitions from the generated model


In [ ]:
from collections import Counter
def transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Most occured transitions:")
transitions(Counter(crf_model.transition_features_).most_common(20))
print("\nMost rearely occuring transitions:")
transitions(Counter(crf_model.transition_features_).most_common()[-20:])

Most occured transitions:
B-org  -> I-org   5.477033
I-org  -> I-org   5.223045
B-per  -> I-per   5.157677
I-per  -> I-per   4.847110
B-geo  -> I-geo   4.220729
B-art  -> I-art   4.216982
B-eve  -> I-eve   3.978056
B-gpe  -> I-gpe   3.866274
O      -> O       3.764479
I-art  -> I-art   3.699787
I-gpe  -> I-gpe   2.809430
I-geo  -> I-geo   2.654627
B-tim  -> I-tim   2.441589
I-tim  -> I-tim   2.428173
O      -> B-tim   2.038565
B-org  -> B-art   2.016776
O      -> B-org   1.988344
O      -> B-gpe   1.985062
I-eve  -> I-eve   1.849034
B-gpe  -> B-per   1.778513

Most rearely occuring transitions:
O      -> B-per   0.767017
B-gpe  -> O       0.593340
O      -> B-nat   0.589724
B-tim  -> B-geo   0.472231
B-geo  -> O       0.377201
I-tim  -> B-geo   0.336166
B-tim  -> O       0.261187
I-org  -> B-tim   0.175477
I-per  -> O       0.122313
I-org  -> B-per   0.086680
I-org  -> O       0.069080
B-org  -> O       -0.021006
B-art  -> O       -0.039118
I-geo  -> O       -0.114306
I-gpe  -> O      

#<b> Weights of the model

In [ ]:
# Let's now inspect model weights

el.show_weights(crf_model, top=30)

# The output shows that B-art is followed by I-article, B-eve is followed by I-eve etc. which is logical.

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)




#<b>1.   Let's do some regularization. c1 is the l1 regularization and c2 is the l2 regularization 
#<b>2.   We can increase the c1 parameter to enforce sparsity in the context 



In [ ]:
crf_model_regularized = crf(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

#<b>Prediction using CRF after regularization 

In [ ]:
prediction = cvp(estimator=crf_model_regularized, X=X, y=y, cv=5)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [ ]:
classification_analysis_reg = fcr(y_pred=prediction, y_true=y)
print(classification_analysis_reg)

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        28
       B-eve       0.00      0.00      0.00        10
       B-geo       0.70      0.76      0.73       244
       B-gpe       0.72      0.76      0.74       303
       B-nat       1.00      0.20      0.33         5
       B-org       0.66      0.63      0.64       176
       B-per       0.78      0.82      0.80       160
       B-tim       0.98      0.90      0.94       149
       I-art       0.00      0.00      0.00        20
       I-eve       0.00      0.00      0.00        10
       I-geo       0.73      0.35      0.48        31
       I-gpe       0.73      0.40      0.52        20
       I-nat       0.00      0.00      0.00         2
       I-org       0.66      0.74      0.69       140
       I-per       0.84      0.96      0.90       206
       I-tim       0.90      0.69      0.78        13
           O       0.99      1.00      1.00      8483

    accuracy              

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
crf_model_regularized.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=10, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [ ]:
el.show_weights(crf_model_regularized, top=30)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
